In [426]:
import pandas as pd
import requests
import json
import xgboost as xgb
from sklearn.metrics import accuracy_score
from datetime import datetime

games_df = pd.read_csv('nhl_games_2019.csv', index_col=0)

In [427]:
games_df = games_df.drop(['gameDate', 'venue', 'gamePk', 'link', 'gameType', 'status', 'teams', 'content', 'season'], axis=1)
games_df = games_df.reset_index()

print (games_df[games_df['Total_Goals'] == 0].index)
games_df = games_df.drop(games_df[games_df['Total_Goals'] == 0].index, axis=0)

games_df = games_df.drop(['index'], axis=1)

Int64Index([2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047,
            ...
            2532, 2533, 2534, 2535, 2536, 2537, 2538, 2539, 2540, 2541],
           dtype='int64', length=504)


In [428]:
games_df

,Away_Team_ID,Home_Team_ID,Away_Team_Wins,Away_Team_Losses,Home_Team_Wins,Home_Team_Losses,Total_Goals
0,8,10,0,0,1,0,5
1,6,15,0,1,1,0,7
2,20,23,0,1,1,0,7
3,24,28,1,0,0,1,7
4,6,7,1,1,0,1,4
...,...,...,...,...,...,...,...
2033,52,12,25,21,29,18,5
2034,5,4,31,14,27,17,3
2035,13,16,28,16,24,21,7
2036,52,29,25,22,27,16,7


In [429]:

#max_depth = 6, min_child_weight = 6, 0.01, 672: 0.494873817035
def get_xgb_params():
    params = {}
    params['objective'] = 'multi:softmax'
    params['silent'] = 0
        
    #params['learning_rate'], params['n_estimators'] = 0.1, 92
    #params['learning_rate'], params['n_estimators'] = 0.05, 209
    #params['learning_rate'], params['n_estimators'] = 0.01, 210
    
    params['learning_rate'], params['n_estimators'] = 0.1, 92
    params['max_depth'], params['min_child_weight'] = 10, 10
    params['gamma'] = 4.3
    params['colsample_bytree'], params['subsample'] = 0.7,0.75
    params['reg_alpha'] = 1e-5
    return params

In [430]:
data_train = games_df[:1000]
data_test = games_df[1000:]

X_train = data_train.drop("Total_Goals", axis=1)
Y_train = data_train.Total_Goals
X_test = data_test.drop("Total_Goals", axis=1)
Y_test = data_test.Total_Goals

In [431]:
classifier = xgb.XGBClassifier(**get_xgb_params())

classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)

print (accuracy_score(y_true=Y_test, y_pred=y_pred))


0.21483622350674375
